In [8]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [9]:
# Data preparation function for categories
def load_and_preprocess_images(directory, image_size=(224, 224)):
    images = []
    labels = []
    paths = []
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".jpg") or file.endswith(".png"):
                img_path = os.path.join(subdir, file)
                image = cv2.imread(img_path)
                if image is not None:
                    image = cv2.resize(image, image_size)
                    image = image / 255.0  # Normalize
                    images.append(image)
                    labels.append(subdir.split(os.sep)[-2])  # Use the parent directory as label
                    paths.append(img_path)
    return np.array(images), labels, paths

# Load images from the provided dataset directory
dataset_dir = 'train'  # Specify your dataset directory here
images, labels, image_paths = load_and_preprocess_images(dataset_dir)
print(f"Loaded {len(images)} images from the dataset.")

Loaded 201 images from the dataset.


In [10]:
# Data preparation function for styles
def load_and_preprocess_images_with_styles(directory, image_size=(224, 224)):
    images = []
    styles = []
    paths = []
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".jpg") or file.endswith(".png"):
                img_path = os.path.join(subdir, file)
                image = cv2.imread(img_path)
                if image is not None:
                    image = cv2.resize(image, image_size)
                    image = image / 255.0  # Normalize
                    images.append(image)
                    styles.append(subdir.split(os.sep)[-1])  # Use the grandparent directory as style
                    paths.append(img_path)
    return np.array(images), styles, paths

# Load images and styles from the provided dataset directory
images, styles, image_paths = load_and_preprocess_images_with_styles(dataset_dir)
print(f"Loaded {len(images)} images with styles from the dataset.")

Loaded 201 images with styles from the dataset.


In [11]:
# Prepare data for training category model
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

# Split data into training and validation sets for category classification
X_train, X_val, y_train, y_val = train_test_split(images, labels_categorical, test_size=0.2, random_state=42)

In [12]:
# Prepare data for training style classification model
style_encoder = LabelEncoder()
styles_encoded = style_encoder.fit_transform(styles)
styles_categorical = to_categorical(styles_encoded)

# Split data into training and validation sets for style classification
X_train_style, X_val_style, y_train_style, y_val_style = train_test_split(images, styles_categorical, test_size=0.2, random_state=42)

In [13]:
# Number of classes and styles
num_classes = len(np.unique(labels))
num_styles = len(np.unique(styles))

In [14]:
# Define and train the enhanced CNN model for category classification
def build_enhanced_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(256, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

cnn_model = build_enhanced_cnn_model((224, 224, 3), num_classes)

E:\Old Files\Machine Learning\New folder\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Train the enhanced CNN model for categories
history = cnn_model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

# Define and train the enhanced CNN model for style classification
style_model = build_enhanced_cnn_model((224, 224, 3), num_styles)

# Train the style classification model
style_model.fit(X_train_style, y_train_style, epochs=20, validation_data=(X_val_style, y_val_style))

# Ensure the model is built by calling it with some input data
cnn_model.predict(np.expand_dims(images[0], axis=0))
style_model.predict(np.expand_dims(images[0], axis=0))

# Create models to extract features from the penultimate layer
feature_extractor = Model(inputs=cnn_model.input, outputs=cnn_model.layers[-2].output)
style_feature_extractor = Model(inputs=style_model.input, outputs=style_model.layers[-2].output)

# Extract features for the entire dataset
features = feature_extractor.predict(images)
style_features = style_feature_extractor.predict(images)
print(f"Extracted features for {len(features)} images.")
print(f"Extracted style features for {len(style_features)} images.")

Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.5735 - loss: 1.2102 - val_accuracy: 0.5366 - val_loss: 0.7703
Epoch 2/20


In [ ]:
# Function to find similar images with matching styles
def find_similar_images_with_style(input_image_path, feature_model, style_model, features, styles, image_paths, top_n=10):
    input_image = cv2.imread(input_image_path)
    input_image = cv2.resize(input_image, (224, 224)) / 255.0
    input_image = np.expand_dims(input_image, axis=0)
    
    # Extract features and style of the input image
    input_features = feature_model.predict(input_image)
    input_style = np.argmax(style_model.predict(input_image), axis=1)[0]
    
    # Find similar images
    similarities = cosine_similarity(input_features, features)
    similar_indices = np.argsort(similarities[0])[::-1]
    
    # Filter similar images by matching style
    matching_style_images = [image_paths[idx] for idx in similar_indices if styles[idx] == input_style][:top_n]
    return matching_style_images

# Example usage
input_image_path = 'test-pic4.jpg'  # Provide the path to an input image
similar_images_with_style = find_similar_images_with_style(input_image_path, feature_extractor, style_model, features, styles, image_paths)

In [ ]:
# Function to display images
def display_images(image_paths):
    plt.figure(figsize=(15, 10))
    for i, img_path in enumerate(image_paths):
        img = cv2.imread(img_path)
        plt.subplot(2, 5, i + 1)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title(os.path.basename(img_path))
    plt.show()

# Display similar images with matching style
display_images(similar_images_with_style)